# GridPath Workshop (UCSB)

Initialization of functionalities

In [ ]:
import os
import pandas as pd
import sys

# Print the current working directory. The result should end with state_model
software_dir = os.getcwd()
os.chdir("..")
print(software_dir)

In [ ]:
#Setting this as the primary directory for all future use
# This is be the main folder where all files are created and stored in. 
directory = os.getcwd()
print(directory)

In [ ]:
# Defining the database directory - where the database file will be created and stored. 
db_dir = f"{directory}/db"

# Defining CSV inputs directory - where the state level input files are stored. 
csv_dir = f"{directory}/input_csvs_gridpath"

#Defining scenario folder location
scen_dir = f"{directory}/scenarios"

# GridPath
The following section is for running GridPath.

## User edit section
Add the folder name with input csvs and provide a name for this database.

In [ ]:
# Speciific  model for this run.
# Add the name of the specific mode folder file by replacing the worth between " " 
input_csvs_gridpath = "TN_7-2050-24"

# Add your database name below between "" or leave blank to use input_csvs_gridpath as database name
db_name = ""
db_name = db_name if db_name else input_csvs_gridpath
print(f"Database name is set to: {db_name}")

## Creating database, importing state inputs and importing scenarios to the database for modelling

In [ ]:
!python {directory}/gridpath/db/create_database.py --database {db_dir}/{db_name}.db
!python {directory}/gridpath/db/utilities/port_csvs_to_db.py --database {db_dir}/{db_name}.db --csv_location {csv_dir}/{input_csvs_gridpath}
!python {directory}/gridpath/db/utilities/scenario.py --database {db_dir}/{db_name}.db --csv_path {csv_dir}/{input_csvs_gridpath}/scenarios.csv

In [ ]:
# This will print out all possible scenarios available
scenario_csv = pd.read_csv(f"{csv_dir}/{input_csvs_gridpath}/scenarios.csv")
column_names = scenario_csv.columns[1:]

# Print scenario names
print("Available scenarios:")
for name in column_names:
    print(name)

## Select scenarios from the list above and add to the selected_scenario below before running.

In [ ]:
### Select scenario #1 from above for run
selected_scenario = "VRElow_STlow_CONVmid_BPHS_NuclearPVboth_45Ret_PRM110_TN_3_2030_24"
!python {directory}/gridpath/gridpath/run_end_to_end.py --database {db_dir}/{db_name}.db --scenario {selected_scenario} --scenario_location {scen_dir}

In [ ]:
### Select scenario #2 from above for run
selected_scenario = "VREmid_STmid_CONVmid_BPHS_NuclearPVboth_45Ret_PRM110_TN_3_2030_24"
!python {directory}/gridpath/gridpath/run_end_to_end.py --database {db_dir}/{db_name}.db --scenario {selected_scenario} --scenario_location {scen_dir}

In [ ]:
### Select scenario #3 from above for run
selected_scenario = "VREhigh_SThigh_CONVmid_BPHS_NuclearPVboth_45Ret_PRM110_TN_3_2030_24"
!python {directory}/gridpath/gridpath/run_end_to_end.py --database {db_dir}/{db_name}.db --scenario {selected_scenario} --scenario_location {scen_dir}

# Vizualization
The following section is for visualizing the results from the GridPath model above.

In [ ]:
import sys
import os
import pandas as pd

# Checking the directory is correct - should be state_model folder
os.chdir(software_dir)
print(os.getcwd())

In [ ]:
# loading packages and setting results folder
from loading_utils import * 
from plotting_utils import * 

path_to_scenarios = f"{directory}/scenarios"
print(path_to_scenarios)

Loading necessary functions for the zones, scenarios labels `<scenario_labels.csv>` and technology colors `<technology_labels.csv>` to plot the graph for new and existing capacity

In [ ]:
scen_labels_ = pd.read_csv(directory + '/input_csvs_plotting/scenario_labels.csv') 
tech_labels_ = pd.read_csv(directory + '/input_csvs_plotting/technology_labels.csv') 

Load new and existing capacity across different scenarios, grouping the capacity by technology.

In [ ]:
capacity_         = _load_new_and_existing_capacity(scen_labels_, path_to_scenarios)
grouped_capacity_ = _group_capacity_technologies(capacity_, tech_labels_)
grouped_capacity_.style

Plotting the new and existing capacity across different scenarios. Add title name for the plot.

In [ ]:
# Enter a title of your choice between the quotation marks ('') under title = ''.
_plot_new_and_existing_capacity(grouped_capacity_, scen_labels_, tech_labels_,
                                title        = '',
                                units_label  = 'Existing & New Capacity (GW)',
                                units        = 1e3,
                                legend       = True,
                                save         = True,
                                file_name    = directory + r'/plots/group_installed_capacity.png',
                                y_grid_inc   = 25)

Load energy dispatch across scenarios and group the energy dispatch by technology

In [ ]:
dispatch_         = _load_energy_dispatch(scen_labels_, path_to_scenarios)
tx_losses_        = _load_energy_transmission(scen_labels_, path_to_scenarios)
dispatch_         = _merge_dispatch_and_tx_losses_(dispatch_, tx_losses_)
grouped_dispatch_ = _group_dispatch_technologies(dispatch_, tech_labels_)
grouped_dispatch_.style

Plotting energy dispatch per technology for different scenarios

In [ ]:
# Enter a title of your choice between the quotation marks ('') under title = ''.
_plot_dispatch(grouped_dispatch_, scen_labels_,  tech_labels_, 
               units        = 1e6, 
               units_label  = r'Electricity Generation (TWh)',
               save         = True,
               legend       = True,
                y_grid_inc  = 50,
               title        = '',
               file_name    = directory + r'/plots/group_electricity_generation.png')

Load GHG emissions and load demand date for different scenarios

In [ ]:
emissions_, demands_ = _load_GHG_emissions(scen_labels_, path_to_scenarios)
emissions_.style

Plot GHG emissions for different scenarios

In [ ]:
# Enter a title of your choice between the quotation marks ('') under title = ''.
_plot_emissions(emissions_, scen_labels_, 
                save       = True,
                legend     = True,
                units      = 1e6,
                unit_label = r'GHG Emissions (MtCO$_2$)',
                title      = '',
                file_name  = directory + r'/plots/GHG_emissions.png')

Processing GHG emissions intensity for different scenarios

In [ ]:
intesity_, demands_ = _GHG_emissions_intensity(emissions_, demands_)
intesity_.style

Plot GHG emissions intensity for different scenarios

In [ ]:
# Enter a title of your choice between the quotation marks ('') under title = ''.
_plot_emissions_intensity(intesity_, scen_labels_, 
                          save       = True,
                          legend     = True,
                          unit_label = r'GHG Intensity (MtCO$_2$/MWh)',
                          title      = '',
                          file_name  = directory + r'/plots/GHG_emissions_intensity.png')

Load system cost data for different scenarios

In [ ]:
system_cost_ = _load_system_cost(scen_labels_, path_to_scenarios)
system_cost_.style

Plot LCOE system cost for different scenarios

In [ ]:
# Enter a title of your choice between the quotation marks ('') under title = ''.
_plot_system_cost(system_cost_, scen_labels_, 
                  save       = True, 
                  legend     = True,
                  unit_label = r'Costs (USD per MWh)', 
                  title      = '', 
                  file_name  = directory + r'/plots/system_cost.png')